In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

In [ ]:
# Step 1: Load the Dataset
df = pd.read_csv('rossmann_store_sales.csv')

# Display the first few rows of the dataset
print(df.head())

In [ ]:
# Step 2: Clean the Data
# Handle missing values
df.fillna(df.mean(), inplace=True)

In [ ]:
# Step 3: Feature Scaling
sales_data = df['Sales'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
sales_data_scaled = scaler.fit_transform(sales_data)

In [ ]:
# Step 4: Create Sequences for LSTM
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

sequence_length = 60
X, y = create_sequences(sales_data_scaled, sequence_length)

In [ ]:
# Step 5: Train-Test Split
X_train, X_test = X[:int(0.8*len(X))], X[int(0.8*len(X)):]
y_train, y_test = y[:int(0.8*len(y))], y[int(0.8*len(y)):]


In [ ]:
# Step 6: Build and Train LSTM Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
# Step 7: Evaluate the Model
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled))
print("Root Mean Squared Error (RMSE):", rmse)